# Code to generate time series for every candidate project area

In [104]:
# Third-party packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5pyd
import rex
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

import h5py
import time
from scipy.interpolate import interp1d
from scipy.spatial import cKDTree
from tqdm.notebook import trange, tqdm

from datetime import datetime, timedelta
import pylab
from matplotlib import rcParams

In [105]:
path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/wave_time_series/wave_data_syn_west_2006_hourly.csv'
wave_data_w_2006 = pd.read_csv(path, index_col='time', parse_dates=True)
wave_data_w_2006 = wave_data_w_2006.rename(columns={'lat':'NREL_lat','lon':'NREL_lon'})
wave_data_w_2006

,loc_id,NREL_lat,NREL_lon,significant_wave_height_m,peak_period_s,capacity_factor
time,,,,,,
2006-01-01 00:00:00+00:00,0.0,48.8641,-125.386,2.257110,12.1150,0.320274
2006-01-01 01:00:00+00:00,0.0,48.8641,-125.386,2.261897,12.1150,0.321820
2006-01-01 02:00:00+00:00,0.0,48.8641,-125.386,2.266683,12.1150,0.323366
2006-01-01 03:00:00+00:00,0.0,48.8641,-125.386,2.271470,12.1150,0.324911
2006-01-01 04:00:00+00:00,0.0,48.8641,-125.386,2.281707,12.1150,0.328217
...,...,...,...,...,...,...
2006-12-31 19:00:00+00:00,699000.0,37.5328,-122.149,0.000000,14.6597,0.000000
2006-12-31 20:00:00+00:00,699000.0,37.5328,-122.149,0.000000,14.6597,0.000000
2006-12-31 21:00:00+00:00,699000.0,37.5328,-122.149,0.000000,14.6597,0.000000


In [106]:
path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/NREL_locs.csv'
NREL_locs = pd.read_csv(path)
NREL_locs

,loc_id,lat,lon
0,0.0,48.8641,-125.386
1,1000.0,35.4732,-126.790
2,2000.0,32.1398,-117.142
3,3000.0,42.6914,-129.579
4,4000.0,36.8383,-127.495
...,...,...,...
695,695000.0,38.1014,-122.474
696,696000.0,38.1029,-122.375
697,697000.0,38.1134,-122.391
698,698000.0,37.5819,-122.212


In [125]:
path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/sites_of_interest.csv'
sites_of_interest = pd.read_csv(path)
sites_of_interest

,,name,state,lat,lon,type,wc_lon,wc_lat,wc_id,es_lon,es_lat,es_id,energy,revenue
0,0,Neah Bay 1,WA,48.276111,-124.767222,sites_of_interest,-124.816,48.3195,121000,-122.692500,48.211944,1186,858.750008,159.206586
1,1,Neah Bay 2,WA,48.188333,-125.072778,sites_of_interest,-125.013,48.1834,52000,-123.011944,47.091111,2268,1112.805291,190.549577
2,2,La Push 1,WA,48.035278,-124.923056,sites_of_interest,-124.906,48.0749,110000,-123.011944,47.091111,2268,1067.813650,182.314020
3,3,La Push 2,WA,47.922500,-124.702778,sites_of_interest,-124.779,47.8957,144000,-123.011944,47.091111,2268,1035.713838,175.133421
4,4,La Push 3,WA,47.820556,-124.576944,sites_of_interest,-124.538,47.8340,211000,-123.011944,47.091111,2268,0.206370,0.020169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Lompoc 8,CA,34.076389,-120.458611,sites_of_interest,-120.454,33.9770,483000,-120.430000,34.530000,3249,1064.008798,192.837783
96,96,Port Hueneme 1,CA,34.070556,-119.185833,sites_of_interest,-119.135,34.0913,422000,-119.168994,34.129718,3941,54.210417,73.900145
97,97,Torrance,CA,33.527222,-118.872500,sites_of_interest,-118.986,33.8157,250000,-118.393894,33.852652,4413,99.903302,109.120758
98,98,Port Hueneme 2,CA,33.283333,-119.613056,sites_of_interest,-120.008,33.5137,297000,-119.168994,34.129718,3941,997.613314,154.487374


In [126]:
path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/NREL_sites_rep_area.csv'
areas = pd.read_csv(path)
areas['switch_id'] = list(range(1300000000, 1300000100))
areas.head(86)

,site,area_km2,cap_MW,lon,lat,loc_id,switch_id
0,1,226.779044,2267.79044,-124.816,48.3195,121000,1300000000
1,2,327.653147,3276.53147,-125.013,48.1834,52000,1300000001
2,3,333.197933,3331.97933,-124.906,48.0749,110000,1300000002
3,4,260.462051,2604.62051,-124.779,47.8957,144000,1300000003
4,5,400.298929,4002.98929,-124.538,47.8340,211000,1300000004
...,...,...,...,...,...,...,...
81,82,472.560446,4725.60446,-122.249,35.7833,311000,1300000081
82,83,332.811196,3328.11196,-121.637,36.0043,583000,1300000082
83,84,409.129230,4091.29230,-121.299,35.6795,602000,1300000083
84,85,437.749682,4377.49682,-121.273,35.4278,452000,1300000084


In [119]:
#output_path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/NREL_sites_rep_area.csv'
#areas.to_csv(output_path,index=False)

In [108]:
site_list = areas['site']
site_list

0       1
1       2
2       3
3       4
4       5
     ... 
95     96
96     97
97     98
98     99
99    100
Name: site, Length: 100, dtype: int64

In [127]:
for id in site_list:
    current_site = areas['loc_id'].iloc[[id-1]]
    site_i = wave_data_w_2006[wave_data_w_2006['loc_id'] == current_site[id-1]]
    site_i['site'] = id
    current_area = areas['area_km2'].iloc[[id-1]]
    site_i['area_km2'] = current_area[id-1]
    current_cap = areas['cap_MW'].iloc[[id-1]]
    site_i['cap_MW'] = current_cap[id-1]
    current_switch_id = areas['switch_id'].iloc[[id-1]]
    site_i['switch_id'] = current_switch_id[id-1]
    if (id == site_list[0]):
        site_time_series = site_i.copy()
    else:
        site_time_series = pd.concat([site_time_series, site_i])
site_time_series

/var/folders/bd/5zkgv6rx32s3b0jx_9373xhc0000gn/T/ipykernel_4322/4033573031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_i['site'] = id
/var/folders/bd/5zkgv6rx32s3b0jx_9373xhc0000gn/T/ipykernel_4322/4033573031.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_i['area_km2'] = current_area[id-1]
/var/folders/bd/5zkgv6rx32s3b0jx_9373xhc0000gn/T/ipykernel_4322/4033573031.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,loc_id,NREL_lat,NREL_lon,significant_wave_height_m,peak_period_s,capacity_factor,site,area_km2,cap_MW,switch_id
time,,,,,,,,,,
2006-01-01 00:00:00+00:00,121000.0,48.3195,-124.816,2.967320,12.115,0.558966,1,226.779044,2267.79044,1300000000
2006-01-01 01:00:00+00:00,121000.0,48.3195,-124.816,3.013243,12.115,0.575774,1,226.779044,2267.79044,1300000000
2006-01-01 02:00:00+00:00,121000.0,48.3195,-124.816,3.059167,12.115,0.592582,1,226.779044,2267.79044,1300000000
2006-01-01 03:00:00+00:00,121000.0,48.3195,-124.816,3.105090,12.115,0.609390,1,226.779044,2267.79044,1300000000
2006-01-01 04:00:00+00:00,121000.0,48.3195,-124.816,3.126107,12.115,0.617082,1,226.779044,2267.79044,1300000000
...,...,...,...,...,...,...,...,...,...,...
2006-12-31 19:00:00+00:00,83000.0,32.5864,-117.365,1.660427,16.126,0.066961,100,434.359610,4343.59610,1300000099
2006-12-31 20:00:00+00:00,83000.0,32.5864,-117.365,1.713903,16.126,0.072765,100,434.359610,4343.59610,1300000099
2006-12-31 21:00:00+00:00,83000.0,32.5864,-117.365,1.767380,16.126,0.078569,100,434.359610,4343.59610,1300000099


In [128]:
wave_colocation = site_time_series.reset_index()
wave_colocation['year'] = wave_colocation['time'].dt.year.map("{:02}".format)
wave_colocation['month'] = wave_colocation['time'].dt.month.map("{:02}".format)
wave_colocation['day'] = wave_colocation['time'].dt.day.map("{:02}".format)
wave_colocation['hour'] = wave_colocation['time'].dt.hour.map("{:02}".format)
wave_colocation_switch = wave_colocation[['switch_id','capacity_factor','site','year','month','day','hour']]
wave_colocation_switch = wave_colocation_switch.rename(columns={'switch_id':'generation_project','capacity_factor':'gen_max_capacity_factor'})
wave_colocation_switch

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1300000000,0.558966,1,2006,01,01,00
1,1300000000,0.575774,1,2006,01,01,01
2,1300000000,0.592582,1,2006,01,01,02
3,1300000000,0.609390,1,2006,01,01,03
4,1300000000,0.617082,1,2006,01,01,04
...,...,...,...,...,...,...,...
875995,1300000099,0.066961,100,2006,12,31,19
875996,1300000099,0.072765,100,2006,12,31,20
875997,1300000099,0.078569,100,2006,12,31,21
875998,1300000099,0.078569,100,2006,12,31,22


In [122]:
output_path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy//SWITCH Files/nati_wave_colocation_cf_v1.csv'
wave_colocation_switch.to_csv(output_path,index=False)

In [129]:
wavev2 = wave_colocation_switch.copy()
wave_colocation['year'] = wave_colocation['year'].apply(str)
wave_colocation['month'] = wave_colocation['month'].apply(str)
wave_colocation['day'] = wave_colocation['day'].apply(str)
wave_colocation['hour'] = wave_colocation['hour'].apply(str)
wavev2

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1300000000,0.558966,1,2006,01,01,00
1,1300000000,0.575774,1,2006,01,01,01
2,1300000000,0.592582,1,2006,01,01,02
3,1300000000,0.609390,1,2006,01,01,03
4,1300000000,0.617082,1,2006,01,01,04
...,...,...,...,...,...,...,...
875995,1300000099,0.066961,100,2006,12,31,19
875996,1300000099,0.072765,100,2006,12,31,20
875997,1300000099,0.078569,100,2006,12,31,21
875998,1300000099,0.078569,100,2006,12,31,22


In [130]:
output_path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/SWITCH Files/nati_wave_colocation_cf_v1.csv'
wavev2.to_csv(output_path,index=False)

In [131]:
path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/SWITCH Files/nati_offshore_wind_colocation_cf_v1.csv'
wind_switch = pd.read_csv(path)
wind_switch

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1300000100,0.049596,1,2019,1,1,0
1,1300000100,0.052631,1,2019,1,1,1
2,1300000100,0.081047,1,2019,1,1,2
3,1300000100,0.085284,1,2019,1,1,3
4,1300000100,0.043921,1,2019,1,1,4
...,...,...,...,...,...,...,...
875995,1300000199,0.000000,100,2019,12,31,19
875996,1300000199,0.000000,100,2019,12,31,20
875997,1300000199,0.004956,100,2019,12,31,21
875998,1300000199,0.065263,100,2019,12,31,22


In [134]:
wind_switch['year'] = wind_switch['year'].map("{:02}".format).apply(str)
wind_switch['month'] = wind_switch['month'].map("{:02}".format).apply(str)
wind_switch['day'] = wind_switch['day'].map("{:02}".format).apply(str)
wind_switch['hour'] = wind_switch['hour'].map("{:02}".format).apply(str)
wind_switch

,generation_project,gen_max_capacity_factor,site,year,month,day,hour
0,1300000100,0.049596,1,2019,01,01,00
1,1300000100,0.052631,1,2019,01,01,01
2,1300000100,0.081047,1,2019,01,01,02
3,1300000100,0.085284,1,2019,01,01,03
4,1300000100,0.043921,1,2019,01,01,04
...,...,...,...,...,...,...,...
875995,1300000199,0.000000,100,2019,12,31,19
875996,1300000199,0.000000,100,2019,12,31,20
875997,1300000199,0.004956,100,2019,12,31,21
875998,1300000199,0.065263,100,2019,12,31,22


In [135]:
output_path='/Users/nataliagonzalez/Documents/UCSD PhD/2022/Research 2022/Wave Energy/SWITCH Files/nati_offshore_wind_colocation_cf_v1.csv'
wind_switch.to_csv(output_path,index=False)